# Vietnam Traffic Analysis - Unified Training Notebook

This notebook trains **3 models** for traffic analysis:

| Task | Model Type | Classes |
|------|------------|--------|
| **Vehicle Detection** | YOLOv8 Object Detection | car, truck, motorcycle, bus, ambulance |
| **Accident Classification** | Image Classification | accident, no_accident |
| **Traffic Jam Classification** | Image Classification | jam, no_jam |

**Requirements:** Kaggle GPU (P100 or T4)

## 1. Setup

In [ ]:
!pip install ultralytics>=8.0.0 -q

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO
import yaml

WORK_DIR = Path('/kaggle/working')
os.chdir(WORK_DIR)

# Create output directories
(WORK_DIR / 'trained_models').mkdir(exist_ok=True)

## 2. Dataset Configuration

**IMPORTANT:** Update these paths to match your Kaggle dataset!

In [ ]:
# ============================================================
# UPDATE THESE PATHS TO YOUR KAGGLE DATASET
# ============================================================
DATASET_ROOT = '/kaggle/input/vietnam-traffic-dataset'

# Dataset paths
VEHICLE_DATASET = f'{DATASET_ROOT}/vehicle_detection/yolo_dataset'
ACCIDENT_DATASET = f'{DATASET_ROOT}/accident_classification'
TRAFFIC_DATASET = f'{DATASET_ROOT}/traffic_classification'
# ============================================================

# Verify datasets exist
for name, path in [('Vehicle', VEHICLE_DATASET), ('Accident', ACCIDENT_DATASET), ('Traffic', TRAFFIC_DATASET)]:
    exists = os.path.exists(path)
    print(f"{name}: {'Found' if exists else 'NOT FOUND'} at {path}")

---
# TASK 1: Vehicle Detection (YOLOv8)
---

In [ ]:
# Create dataset.yaml for vehicle detection
vehicle_config = {
    'path': VEHICLE_DATASET,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 5,
    'names': {0: 'car', 1: 'truck', 2: 'motorcycle', 3: 'bus', 4: 'ambulance'}
}

vehicle_yaml = WORK_DIR / 'vehicle_dataset.yaml'
with open(vehicle_yaml, 'w') as f:
    yaml.dump(vehicle_config, f)

print(f"Vehicle detection config saved to: {vehicle_yaml}")

In [ ]:
# Train Vehicle Detection Model
print("="*60)
print("TRAINING: Vehicle Detection (YOLOv8)")
print("="*60)

vehicle_model = YOLO('yolov8l.pt')

vehicle_results = vehicle_model.train(
    data=str(vehicle_yaml),
    epochs=100,
    batch=16,
    imgsz=640,
    patience=20,
    device=0,
    project='vietnam_traffic',
    name='vehicle_detection',
    exist_ok=True,
    # Augmentation for traffic
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

print("\nVehicle Detection Training Complete!")

In [ ]:
# Validate vehicle detection model
vehicle_best = YOLO('vietnam_traffic/vehicle_detection/weights/best.pt')
vehicle_val = vehicle_best.val(data=str(vehicle_yaml))

print(f"\nVehicle Detection Results:")
print(f"  mAP50: {vehicle_val.box.map50:.4f}")
print(f"  mAP50-95: {vehicle_val.box.map:.4f}")

---
# TASK 2: Accident Classification
---

In [ ]:
# Check accident dataset
print("Accident Classification Dataset:")
for split in ['train', 'val']:
    split_dir = Path(ACCIDENT_DATASET) / split
    if split_dir.exists():
        for cls in ['accident', 'no_accident']:
            cls_dir = split_dir / cls
            if cls_dir.exists():
                count = len(list(cls_dir.glob('*')))
                print(f"  {split}/{cls}: {count} images")

In [ ]:
# Train Accident Classification Model
print("="*60)
print("TRAINING: Accident Classification (YOLOv8-cls)")
print("="*60)

# Use YOLOv8 classification model
accident_model = YOLO('yolov8l-cls.pt')

accident_results = accident_model.train(
    data=ACCIDENT_DATASET,
    epochs=50,
    batch=32,
    imgsz=224,
    patience=15,
    device=0,
    project='vietnam_traffic',
    name='accident_classification',
    exist_ok=True,
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
)

print("\nAccident Classification Training Complete!")

In [ ]:
# Validate accident classification model
accident_best = YOLO('vietnam_traffic/accident_classification/weights/best.pt')
accident_val = accident_best.val(data=ACCIDENT_DATASET)

print(f"\nAccident Classification Results:")
print(f"  Top-1 Accuracy: {accident_val.top1:.4f}")
print(f"  Top-5 Accuracy: {accident_val.top5:.4f}")

---
# TASK 3: Traffic Jam Classification
---

In [ ]:
# Check traffic jam dataset
print("Traffic Jam Classification Dataset:")
for split in ['train', 'val']:
    split_dir = Path(TRAFFIC_DATASET) / split
    if split_dir.exists():
        for cls in ['jam', 'no_jam']:
            cls_dir = split_dir / cls
            if cls_dir.exists():
                count = len(list(cls_dir.glob('*')))
                print(f"  {split}/{cls}: {count} images")

In [ ]:
# Train Traffic Jam Classification Model
print("="*60)
print("TRAINING: Traffic Jam Classification (YOLOv8-cls)")
print("="*60)

traffic_model = YOLO('yolov8l-cls.pt')

traffic_results = traffic_model.train(
    data=TRAFFIC_DATASET,
    epochs=50,
    batch=32,
    imgsz=224,
    patience=15,
    device=0,
    project='vietnam_traffic',
    name='traffic_classification',
    exist_ok=True,
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
)

print("\nTraffic Jam Classification Training Complete!")

In [ ]:
# Validate traffic jam classification model
traffic_best = YOLO('vietnam_traffic/traffic_classification/weights/best.pt')
traffic_val = traffic_best.val(data=TRAFFIC_DATASET)

print(f"\nTraffic Jam Classification Results:")
print(f"  Top-1 Accuracy: {traffic_val.top1:.4f}")
print(f"  Top-5 Accuracy: {traffic_val.top5:.4f}")

---
# 4. Export All Models
---

In [ ]:
# Copy all trained models to output directory
output_dir = WORK_DIR / 'trained_models'

models_to_copy = [
    ('vietnam_traffic/vehicle_detection/weights/best.pt', 'vehicle_detection_yolov8l.pt'),
    ('vietnam_traffic/accident_classification/weights/best.pt', 'accident_classification_yolov8l.pt'),
    ('vietnam_traffic/traffic_classification/weights/best.pt', 'traffic_classification_yolov8l.pt'),
]

for src, dst in models_to_copy:
    src_path = Path(src)
    if src_path.exists():
        shutil.copy2(src_path, output_dir / dst)
        print(f"Copied: {dst}")
    else:
        print(f"Not found: {src}")

In [ ]:
# Export to ONNX for deployment
print("\nExporting to ONNX format...")

for model_name in ['vehicle_detection_yolov8l.pt', 'accident_classification_yolov8l.pt', 'traffic_classification_yolov8l.pt']:
    model_path = output_dir / model_name
    if model_path.exists():
        model = YOLO(str(model_path))
        onnx_path = model.export(format='onnx', imgsz=640 if 'vehicle' in model_name else 224)
        print(f"  Exported: {model_name} -> ONNX")

In [ ]:
# Create zip file for download
import zipfile

zip_path = WORK_DIR / 'vietnam_traffic_models.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in output_dir.iterdir():
        zipf.write(file, file.name)
        
    # Add training results images
    for task in ['vehicle_detection', 'accident_classification', 'traffic_classification']:
        results_png = Path(f'vietnam_traffic/{task}/results.png')
        if results_png.exists():
            zipf.write(results_png, f'{task}_results.png')

print(f"\nAll models zipped to: {zip_path}")
print(f"Size: {zip_path.stat().st_size / (1024*1024):.1f} MB")

---
# 5. Summary
---

In [ ]:
print("="*60)
print("TRAINING COMPLETE - SUMMARY")
print("="*60)

print("\n1. VEHICLE DETECTION (YOLOv8 Object Detection)")
print(f"   Classes: car, truck, motorcycle, bus, ambulance")
print(f"   Model: vehicle_detection_yolov8l.pt")

print("\n2. ACCIDENT CLASSIFICATION")
print(f"   Classes: accident, no_accident")
print(f"   Model: accident_classification_yolov8l.pt")

print("\n3. TRAFFIC JAM CLASSIFICATION")
print(f"   Classes: jam, no_jam")
print(f"   Model: traffic_classification_yolov8l.pt")

print("\n" + "="*60)
print("Download 'vietnam_traffic_models.zip' from Output tab!")
print("="*60)

## Integration Instructions

After downloading, integrate into your project:

```python
from ultralytics import YOLO

# Load models
vehicle_model = YOLO('vehicle_detection_yolov8l.pt')
accident_model = YOLO('accident_classification_yolov8l.pt')
traffic_model = YOLO('traffic_classification_yolov8l.pt')

# Vehicle detection
detections = vehicle_model(frame)

# Accident classification
accident_result = accident_model(frame)
is_accident = accident_result[0].probs.top1 == 0  # 0 = accident class

# Traffic jam classification  
traffic_result = traffic_model(frame)
is_jam = traffic_result[0].probs.top1 == 0  # 0 = jam, 1 = no_jam
```